In [ ]:
import pandas as pd
import kagglehub
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
#2023 fifa data
# Download latest version
path3 = kagglehub.dataset_download("sanjeetsinghnaik/fifa-23-players-dataset")

print("Path to dataset files:", path3)

Path to dataset files: /kaggle/input/fifa-23-players-dataset


In [ ]:
#2022 fifa data
# Download FIFA 22 dataset from Kaggle
path2 = kagglehub.dataset_download("stefanoleone992/fifa-22-complete-player-dataset")
print("✅ Downloaded to:", path2)

✅ Downloaded to: /kaggle/input/fifa-22-complete-player-dataset


In [ ]:
#2021 fifa data
# Download latest version
path1 = kagglehub.dataset_download("stefanoleone992/fifa-21-complete-player-dataset")

print("Path to dataset files:", path1)

Path to dataset files: /kaggle/input/fifa-21-complete-player-dataset


In [ ]:
#2024 fifa data

# Download latest version
path = kagglehub.dataset_download("nyagami/fc-24-players-database-and-stats-from-easports")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fc-24-players-database-and-stats-from-easports


In [ ]:
df_fifa21 = pd.read_csv("/kaggle/input/fifa-21-complete-player-dataset/players_21.csv")


In [ ]:
df_fifa24 = pd.read_csv("/kaggle/input/fc-24-players-database-and-stats-from-easports/male_fc_24_players.csv")


In [ ]:
#2022
df_fifa22 = pd.read_csv(f"{path2}/players_22.csv")


<ipython-input-506-62050ff24457>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df_fifa22 = pd.read_csv(f"{path2}/players_22.csv")


In [ ]:
#2023
df_fifa23 = pd.read_csv(f"{path3}/Fifa 23 Players Data.csv")

In [ ]:


#rename raw columns -> our standard names per season
col_maps = {
    2021: {'short_name':'player_name','club_name':'club','overall':'overall',
           'pace':'pac','shooting':'sho','passing':'pas','dribbling':'dri',
           'defending':'def','physic':'phy','age':'age'},
    2022: {'short_name':'player_name','club_name':'club','overall':'overall',
           'pace':'pac','shooting':'sho','passing':'pas','dribbling':'dri',
           'defending':'def','physic':'phy','age':'age'},
    2023: {'Known As':'player_name','Club Name':'club','Overall':'overall',
           'Pace Total':'pac','Shooting Total':'sho','Passing Total':'pas',
           'Dribbling Total':'dri','Defending Total':'def',
           'Physicality Total':'phy','Age':'age'},
    2024: {'name':'player_name','club':'club','overall':'overall',
           'PAC':'pac','SHO':'sho','PAS':'pas','DRI':'dri','DEF':'def',
           'PHY':'phy','age':'age'}
}

#all of UCL clubs participating per season
ucl_teams = {
    2021: [ "Bayern Munich", "Atlético Madrid", "Red Bull Salzburg", "Lokomotiv Moscow",
    "Real Madrid", "Shakhtar Donetsk", "Inter Milan", "Borussia Mönchengladbach",
    "Manchester City", "Porto", "Olympiacos", "Marseille",
    "Liverpool", "Atalanta", "Ajax", "Midtjylland",
    "Sevilla", "Chelsea", "Krasnodar", "Rennes",
    "Zenit Saint Petersburg", "Borussia Dortmund", "Lazio", "Club Brugge",
    "Juventus", "Barcelona", "Dynamo Kyiv", "Ferencváros",
    "Paris Saint-Germain", "Manchester United", "RB Leipzig", "İstanbul Başakşehir"],
    2022: [ "Manchester City", "Paris Saint-Germain", "RB Leipzig", "Club Brugge",
    "Liverpool", "Atletico Madrid", "Porto", "AC Milan",
    "Ajax", "Sporting CP", "Borussia Dortmund", "Beşiktaş",
    "Real Madrid", "Inter Milan", "Shakhtar Donetsk", "Sheriff Tiraspol",
    "Bayern Munich", "Benfica", "Barcelona", "Dynamo Kyiv",
    "Manchester United", "Villarreal", "Atalanta", "Young Boys",
    "Lille", "Sevilla", "Red Bull Salzburg", "Wolfsburg",
    "Juventus", "Chelsea", "Zenit Saint Petersburg", "Malmö FF" ],
    2023: [ "Real Madrid", "RB Leipzig", "Shakhtar Donetsk", "Celtic",
    "Manchester City", "Borussia Dortmund", "Sevilla", "Copenhagen",
    "Paris Saint-Germain", "Benfica", "Juventus", "Maccabi Haifa",
    "Chelsea", "AC Milan", "Red Bull Salzburg", "Dinamo Zagreb",
    "Napoli", "Liverpool", "Ajax", "Rangers",
    "Porto", "Club Brugge", "Bayer Leverkusen", "Atlético Madrid",
    "Bayern Munich", "Inter", "Barcelona", "Viktoria Plzeň",
    "Tottenham Hotspur", "Eintracht Frankfurt", "Sporting CP", "Marseille"],
    2024: [ "Manchester City", "Real Madrid", "Bayern Munich", "Barcelona", "Arsenal",
    "Paris SG", "Napoli", "Inter", "AC Milan", "Atletico Madrid", "Borussia Dortmund",
    "RB Leipzig", "Porto", "Benfica", "Sevilla", "Celtic", "Galatasaray",
    "Feyenoord", "PSV", "Lens", "Union Berlin", "Royal Antwerp", "Braga",
    "Copenhagen", "Young Boys", "Shakhtar Donetsk", "Red Star Belgrade" ]
}

#true winners for every season
winners = {
    2021: "Chelsea",
    2022: "Real Madrid",
    2023: "Manchester City",
    2024: "Real Madrid"
}

#feature names your 2023–24 model expects:
feat_cols = X.columns.tolist()  # from your existing X


In [ ]:
#raw df_fifa21 already loaded    featues for ucl teams, did it for the other 3 seasons included in the model
df21 = df_fifa21.rename(columns=col_maps[2021])[ list(col_maps[2021].values()) ]

#filter UCL clubs
df21 = df21[df21['club'].isin(ucl_teams[2021])]

#build features
stats = ['overall','pac','sho','pas','dri','def','phy','age']
avg21 = df21.groupby('club')[stats].mean().add_prefix('avg_')
top5_21 = (df21.sort_values('overall',ascending=False)
             .groupby('club').head(5)
             .groupby('club')[stats].mean()
             .add_prefix('top5_avg_'))
cnt21=df21.groupby('club').size().rename('num_players')

team_features_2021= pd.concat([avg21,top5_21, cnt21], axis=1)
team_features_2021['label'] = (team_features_2021.index == winners[2021]).astype(int)
team_features_2021.index.name = 'club'
team_features_2021 = team_features_2021.reset_index()


In [ ]:
#raw df_fifa22 already loaded
df22 = df_fifa22.rename(columns=col_maps[2022])[ list(col_maps[2022].values()) ]
df22 = df22[df22['club'].isin(ucl_teams[2022])]

avg22 = df22.groupby('club')[stats].mean().add_prefix('avg_')
top5_22 = (df22.sort_values('overall',ascending=False)
             .groupby('club').head(5)
             .groupby('club')[stats].mean()
             .add_prefix('top5_avg_'))
cnt22 = df22.groupby('club').size().rename('num_players')

team_features_2022 = pd.concat([avg22, top5_22, cnt22], axis=1)
team_features_2022['label'] = (team_features_2022.index == winners[2022]).astype(int)
team_features_2022.index.name = 'club'
team_features_2022 = team_features_2022.reset_index()


In [ ]:
#raw df_fifa23 already loaded
df23 = df_fifa23.rename(columns=col_maps[2023])[ list(col_maps[2023].values()) ]
df23 = df23[df23['club'].isin(ucl_teams[2023])]

avg23 = df23.groupby('club')[stats].mean().add_prefix('avg_')
top5_23 = (df23.sort_values('overall',ascending=False)
             .groupby('club').head(5)
             .groupby('club')[stats].mean()
             .add_prefix('top5_avg_'))
cnt23 = df23.groupby('club').size().rename('num_players')

team_features_2023 = pd.concat([avg23, top5_23, cnt23], axis=1)
team_features_2023['label'] = (team_features_2023.index == winners[2023]).astype(int)
team_features_2023.index.name = 'club'
team_features_2023 = team_features_2023.reset_index()


In [ ]:
#raw df_fifa24 already loaded
df24 = df_fifa24.rename(columns=col_maps[2024])[ list(col_maps[2024].values()) ]
df24 = df24[df24['club'].isin(ucl_teams[2024])]

avg24 = df24.groupby('club')[stats].mean().add_prefix('avg_')
top5_24 = (df24.sort_values('overall',ascending=False)
             .groupby('club').head(5)
             .groupby('club')[stats].mean()
             .add_prefix('top5_avg_'))
cnt24 = df24.groupby('club').size().rename('num_players')

team_features_2024 = pd.concat([avg24, top5_24, cnt24], axis=1)
team_features_2024['label'] = (team_features_2024.index == winners[2024]).astype(int)
team_features_2024.index.name = 'club'
team_features_2024 = team_features_2024.reset_index()


In [ ]:
#feat_cols:models exact feature names from 2023–24 training
feat_cols =X.columns.tolist()
#Function to align column names of any dataframe to match the training features
def align_columns(df):
    rename_map = {}
    for col in df.columns:
        low = col.lower()
        #find thee matching feature name
        match = next((f for f in feat_cols if f.lower() == low), None)
        if match and col != match:
            rename_map[col] = match
    df.rename(columns=rename_map, inplace=True)

#apply to each season
for df in (team_features_2021, team_features_2022, team_features_2023,team_features):
    align_columns(df)




In [ ]:
print("2021 now has:", sorted(team_features_2021.columns))
print("Expecting   :", sorted(feat_cols + ['club','label']))




2021 now has: ['avg_DEF', 'avg_DRI', 'avg_PAC', 'avg_PAS', 'avg_PHY', 'avg_SHO', 'avg_age', 'avg_overall', 'club', 'label', 'num_players', 'top5_avg_DEF', 'top5_avg_DRI', 'top5_avg_PAC', 'top5_avg_PAS', 'top5_avg_PHY', 'top5_avg_SHO', 'top5_avg_age', 'top5_avg_overall']
Expecting   : ['avg_DEF', 'avg_DRI', 'avg_PAC', 'avg_PAS', 'avg_PHY', 'avg_SHO', 'avg_age', 'avg_overall', 'club', 'label', 'num_players', 'top5_avg_DEF', 'top5_avg_DRI', 'top5_avg_PAC', 'top5_avg_PAS', 'top5_avg_PHY', 'top5_avg_SHO', 'top5_avg_age', 'top5_avg_overall']


In [ ]:


from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

#stack all four seasons
#combine features from all four seasons into one dataset
df_all = pd.concat(
    [team_features_2021, team_features_2022, team_features_2023, team_features],
    ignore_index=True
)
X_all = df_all.drop(columns=['club','label'])[feat_cols]
y_all = df_all['label']

#Hyperparameter search space
param_dist = {
    'n_estimators':      [100, 200, 500, 1000],
    'max_depth':         [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf':  [1, 2, 4],
    'class_weight':      ['balanced', 'balanced_subsample']
}


rs = RandomizedSearchCV(
    RandomForestClassifier(random_state=42),
    param_dist,
    n_iter=34,
    cv=4,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42
)
rs.fit(X_all, y_all)

print("Best forest params:", rs.best_params_)
best_model = rs.best_estimator_

#Now report per season predictions with the tuned model
actual = {2021:"Chelsea", 2022:"Real Madrid", 2023:"Manchester City", 2024:"Real Madrid"}
for year, df in [(2021, team_features_2021),
                 (2022, team_features_2022),
                 (2023, team_features_2023),
                 (2024, team_features)]:
    X_year = df.drop(columns=['club','label'])[feat_cols]
    probs   = best_model.predict_proba(X_year)[:,1]
    pred    = df['club'].iloc[probs.argmax()]
    print(f"Season {year}: predicted → {pred:15}   actual → {actual[year]}")





/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=4.
  warnings.warn(


Best forest params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 10, 'class_weight': 'balanced_subsample'}
Season 2021: predicted → Chelsea           actual → Chelsea
Season 2022: predicted → Juventus          actual → Real Madrid
Season 2023: predicted → Manchester City   actual → Manchester City
Season 2024: predicted → Real Madrid       actual → Real Madrid


In [ ]:
#lean FIFA 24 using the uppercase stat names
fifa24_cleaned = df_fifa24[[
    'club',
    'overall',
    'PAC', 'SHO', 'PAS', 'DRI', 'DEF', 'PHY',
    'age'
]].rename(columns={
    'overall': 'ovr',
    'PAC':      'pac',
    'SHO':      'sho',
    'PAS':      'pas',
    'DRI':      'dri',
    'DEF':      'def',
    'PHY':      'phy'
})

#use that as df25
df25 = fifa24_cleaned

In [ ]:
#2025 fifa data
# Download latest version
path5 = kagglehub.dataset_download("akshyakumarkc/fifa-25-player-ratings")

print("Path to dataset files:", path5)

Path to dataset files: /kaggle/input/fifa-25-player-ratings


In [ ]:
df_fifa25 = pd.read_csv(f"{path5}/players_info.csv")


In [ ]:

#keep only males
df25_male = df_fifa25[df_fifa25['gender'] == 'M']

#select just the columns our feature builder expects
df25 = df25_male[[
    'club',
    'ovr',   #overall rating
    'pac',   #pace
    'sho',   # ssshooting
    'pas',   #passing
    'dri',   #dribbling
    'def',   #defending
    'phy',   #physicality
    'age'
]]
semis_2025 = [   #teams that were in semi finals this season
    "FC Barcelona",
    "Arsenal",
    "Paris SG",
    "Lombardia FC"   #Inter Milan5 name in fifa
]



['rank', 'player_name', 'age', 'gender', 'club', 'league', 'nationality', 'position', 'preffered_foot', 'height_(in cm)', 'weight_(in kg)', 'alt_position', 'ovr', 'pac', 'pac+-', 'sho', 'sho+-', 'pas', 'pas+-', 'dri', 'dri+-', 'def', 'def+-', 'phy', 'phy+-', 'image_url']


In [ ]:
tf_semis = build_team_features(
    df=df25,
    teams=semis_2025,
    champion=None    #we don’t supply a label here
)





In [ ]:
tf_semis.rename(columns={
    'avg_ovr':       'avg_overall',
    'top5_avg_ovr':  'top5_avg_overall'
}, inplace=True)
align_columns(tf_semis)


In [ ]:
#to standardize column names across datasets
def align_columns(df):
    rename_map = {}
    for col in df.columns:
        low = col.lower()
        match = next((f for f in feat_cols if f.lower() == low), None)
        if match and col != match:
            rename_map[col] = match
    df.rename(columns=rename_map, inplace=True)


In [ ]:
print("Semis columns:", sorted(tf_semis.columns))
print("Expecting   :", sorted(feat_cols + ['club','label']))


Semis columns: ['avg_DEF', 'avg_DRI', 'avg_PAC', 'avg_PAS', 'avg_PHY', 'avg_SHO', 'avg_age', 'avg_overall', 'club', 'num_players', 'top5_avg_DEF', 'top5_avg_DRI', 'top5_avg_PAC', 'top5_avg_PAS', 'top5_avg_PHY', 'top5_avg_SHO', 'top5_avg_age', 'top5_avg_overall']
Expecting   : ['avg_DEF', 'avg_DRI', 'avg_PAC', 'avg_PAS', 'avg_PHY', 'avg_SHO', 'avg_age', 'avg_overall', 'club', 'label', 'num_players', 'top5_avg_DEF', 'top5_avg_DRI', 'top5_avg_PAC', 'top5_avg_PAS', 'top5_avg_PHY', 'top5_avg_SHO', 'top5_avg_age', 'top5_avg_overall']


In [ ]:
#extract model inputs in the correct order
X_semis = tf_semis[feat_cols]

#get each team’s probability of winning
probs_semis = final_model_all.predict_proba(X_semis)[:, 1]

#club with the highest probability
pick_semis = tf_semis['club'].iloc[probs_semis.argmax()]

print("Predicted Champion from Semifinals:", pick_semis)
print("Confidence:", probs_semis.max())


🔮 Predicted Champion from Semifinals: Lombardia FC
🏅 Confidence: 0.35


chelsea
rma
mci
rma